# **LP2_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are in charge of the supply purchasing of a company that produces two kinds of drugs.
The drugs contain a specific active agent, which is extracted from two different kinds of raw materials that should be purchased on the market.

The goal is to maximize the total profit obtained from producing the drugs, which means minimizing purchasing costs for buying the raw materials and minimizing operational costs for producing the drugs while maximizing sales.

You are given a budget for purchasing raw materials and operating the production process of the drugs which cannot be exceeded.
Additionally, you need to keep the capacity constraints for the production of the drugs in mind as there is only a limited amount of manpower as well as equipment hours available.
Your company can also only store a limited amount of raw materials.
Most importantly, the amount of active ingredient extracted from the raw materials you purchase needs to satisfy the required amount of active ingredient in the drugs your company produces.
Importantly, your experience with the production of the drugs has shown the the amount of active ingredient your process extracts from the two raw materials can vary by up to 0.5% and 2%, respectively.
During purchasing, you need to make sure that no matter how much the amount of active ingredient varies, the required amount for the drug production is always met. """

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [6]:
print(response1.choices[0].message.content)

1. X1, X2: Quantity of raw material 1 and 2 to be purchased.
2. D1, D2: Quantity of drug 1 and drug 2 to be produced.
3. A1, A2: Amount of active ingredient extracted from raw material 1 and 2.
4. P1, P2: Selling price of drug 1 and drug 2.
5. C1, C2: Purchase cost of raw material 1 and 2.
6. O1, O2: Operation cost for producing drug 1 and drug 2.
7. M: Manpower for producing the drugs.
8. E: Equipment hours available for producing the drugs.
9. S: Storage capacity for the raw materials.
10. B: Budget for purchasing raw materials and operation of drug production process.
11. R1, R2: Required amount of active ingredient in drug 1 and drug 2.
12. V1, V2: Variation in the amount of active ingredient extracted from raw material 1 and 2. 
13. T1, T2: Total active ingredient needed to produce drug 1 and drug 2 considering the variation.


In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [8]:
print(response2.choices[0].message.content)

Maximize Z = P1*D1 + P2*D2 - C1*X1 - C2*X2 - O1*D1 - O2*D2


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [10]:
print(response3.choices[0].message.content)

Constraints:

1. Budget constraint: C1*X1 + C2*X2 + O1*D1 + O2*D2 <= B
2. Manpower constraint: M >= D1 + D2
3. Equipment hours constraint: E >= D1 + D2
4. Storage capacity constraint: X1 + X2 <= S
5. Required amount of active ingredient for drug 1 considering the variation: T1 = D1*R1*(1+V1)
6. Required amount of active ingredient for drug 2 considering the variation: T2 = D2*R2*(1+V2)
7. Amount of active ingredient extracted from raw material 1 and 2 should be more than or equal to the total required amount of active ingredient considering variation: A1*X1 + A2*X2 >= T1 + T2
8. Non-negativity constraints:
   X1, X2, D1, D2, A1, A2, P1, P2, C1, C2, O1, O2, M, E, S, B, R1, R2, V1, V2, T1, T2 >= 0



### **Generate Pyomo Code**

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [12]:
Markdown(response4.choices[0].message.content)

Sure. Here you go:

```python
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Define variables
model.X1 = Var(domain=NonNegativeReals)
model.X2 = Var(domain=NonNegativeReals)
model.D1 = Var(domain=NonNegativeReals)
model.D2 = Var(domain=NonNegativeReals)
model.A1 = Var(domain=NonNegativeReals)
model.A2 = Var(domain=NonNegativeReals)

# Define parameters (sample data is used)
model.P1 = Param(initialize=20)
model.P2 = Param(initialize=30)
model.C1 = Param(initialize=10)
model.C2 = Param(initialize=15)
model.O1 = Param(initialize=3)
model.O2 = Param(initialize=4)
model.M = Param(initialize=100)
model.E = Param(initialize=50)
model.S = Param(initialize=60)
model.B = Param(initialize=1000)
model.R1 = Param(initialize=0.1)
model.R2 = Param(initialize=0.2)
model.V1 = Param(initialize=0.05)
model.V2 = Param(initialize=0.1)

# Objective
model.obj = Objective(expr = model.P1*model.D1 + model.P2*model.D2 - model.C1*model.X1 - model.C2*model.X2 - model.O1*model.D1 - model.O2*model.D2, sense=maximize)

# Constraints
model.budget = Constraint(expr = model.C1*model.X1 + model.C2*model.X2 + model.O1*model.D1 + model.O2*model.D2 <= model.B)
model.manpower = Constraint(expr = model.M >= model.D1 + model.D2)
model.equipment = Constraint(expr = model.E >= model.D1 + model.D2)
model.storage = Constraint(expr = model.X1 + model.X2 <= model.S)
model.ingredient1 = Constraint(expr = model.D1*model.R1*(1+model.V1) <= model.A1*model.X1) 
model.ingredient2 = Constraint(expr = model.D2*model.R2*(1+model.V2) <= model.A2*model.X2) 
model.total_ingredient = Constraint(expr = model.A1*model.X1 + model.A2*model.X2 >= model.D1*model.R1*(1+model.V1) + model.D2*model.R2*(1+model.V2)) 

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# Print the values of the decision variables
print("Values of decision variables:")
print("X1 = ", model.X1.value)
print("X2 = ", model.X2.value)
print("D1 = ", model.D1.value)
print("D2 = ", model.D2.value)
print("A1 = ", model.A1.value)
print("A2 = ", model.A2.value)
```

Pyomo is a Python environment that's part of the Python Optimization Modeling Objects (Pyomo) project. It is utilized in a variety of contexts, such as script-based applications, APIs, etc. SolverFactory is a class in Pyomo which can be used to specify a solver. In the example, 'glpk' solver is used, but you can choose other solvers as per your requirement and based on which solvers are installed on your machine. Additionally, the initializer values of the parameters (P1, P2, C1, C2, O1, O2, M, E, S, B, R1, R2, V1, V2) are given random values for the sake of example and therefore are considered as sample data in this case.

### **Run the code Generated by GPT4**

In [13]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Define variables
model.X1 = Var(domain=NonNegativeReals)
model.X2 = Var(domain=NonNegativeReals)
model.D1 = Var(domain=NonNegativeReals)
model.D2 = Var(domain=NonNegativeReals)
model.A1 = Var(domain=NonNegativeReals)
model.A2 = Var(domain=NonNegativeReals)

# Define parameters (sample data is used)
model.P1 = Param(initialize=20)
model.P2 = Param(initialize=30)
model.C1 = Param(initialize=10)
model.C2 = Param(initialize=15)
model.O1 = Param(initialize=3)
model.O2 = Param(initialize=4)
model.M = Param(initialize=100)
model.E = Param(initialize=50)
model.S = Param(initialize=60)
model.B = Param(initialize=1000)
model.R1 = Param(initialize=0.1)
model.R2 = Param(initialize=0.2)
model.V1 = Param(initialize=0.05)
model.V2 = Param(initialize=0.1)

# Objective
model.obj = Objective(expr = model.P1*model.D1 + model.P2*model.D2 - model.C1*model.X1 - model.C2*model.X2 - model.O1*model.D1 - model.O2*model.D2, sense=maximize)

# Constraints
model.budget = Constraint(expr = model.C1*model.X1 + model.C2*model.X2 + model.O1*model.D1 + model.O2*model.D2 <= model.B)
model.manpower = Constraint(expr = model.M >= model.D1 + model.D2)
model.equipment = Constraint(expr = model.E >= model.D1 + model.D2)
model.storage = Constraint(expr = model.X1 + model.X2 <= model.S)
model.ingredient1 = Constraint(expr = model.D1*model.R1*(1+model.V1) <= model.A1*model.X1)
model.ingredient2 = Constraint(expr = model.D2*model.R2*(1+model.V2) <= model.A2*model.X2)
model.total_ingredient = Constraint(expr = model.A1*model.X1 + model.A2*model.X2 >= model.D1*model.R1*(1+model.V1) + model.D2*model.R2*(1+model.V2))

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# Print the values of the decision variables
print("Values of decision variables:")
print("X1 = ", model.X1.value)
print("X2 = ", model.X2.value)
print("D1 = ", model.D1.value)
print("D2 = ", model.D2.value)
print("A1 = ", model.A1.value)
print("A2 = ", model.A2.value)

ValueError: Model constraint (ingredient1) contains nonlinear terms that cannot be written to LP format

In [17]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 19.4 MB/s eta 0:00:00


In [18]:

_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [23]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Define variables
model.X1 = Var(domain=NonNegativeReals)
model.X2 = Var(domain=NonNegativeReals)
model.D1 = Var(domain=NonNegativeReals)
model.D2 = Var(domain=NonNegativeReals)
model.A1 = Var(domain=NonNegativeReals)
model.A2 = Var(domain=NonNegativeReals)

# Define parameters (sample data is used)
model.P1 = Param(initialize=6200)
model.P2 = Param(initialize=6900)
model.C1 = Param(initialize=100)
model.C2 = Param(initialize=199.90)
model.O1 = Param(initialize=700)
model.O2 = Param(initialize=800)
model.M = Param(initialize=2000)
model.E = Param(initialize=800)
model.S = Param(initialize=1000)
model.B = Param(initialize=100000)
model.R1 = Param(initialize=0.5)
model.R2 = Param(initialize=0.6)
model.V1 = Param(initialize=0.005)
model.V2 = Param(initialize=0.02)

# Objective
model.obj = Objective(expr = model.P1*model.D1 + model.P2*model.D2 - model.C1*model.X1 - model.C2*model.X2 - model.O1*model.D1 - model.O2*model.D2, sense=maximize)

# Constraints
model.budget = Constraint(expr = model.C1*model.X1 + model.C2*model.X2 + model.O1*model.D1 + model.O2*model.D2 <= model.B)
model.manpower = Constraint(expr = model.M >= model.D1 + model.D2)
model.equipment = Constraint(expr = model.E >= model.D1 + model.D2)
model.storage = Constraint(expr = model.X1 + model.X2 <= model.S)
model.ingredient1 = Constraint(expr = model.D1*model.R1 + (model.D1*model.R1)*model.V1 <= model.A1*model.X1)
model.ingredient2 = Constraint(expr = model.D2*model.R2 + (model.D2*model.R2)*model.V2 <= model.A2*model.X2)
model.total_ingredient = Constraint(expr = model.A1*model.X1 + model.A2*model.X2 >= model.D1*model.R1*(1+model.V1) + model.D2*model.R2*(1+model.V2))

# Solve
solver = SolverFactory('knitro')
solver.solve(model)

# Print the values of the decision variables
print("Values of decision variables:")
print("X1 = ", model.X1.value)
print("X2 = ", model.X2.value)
print("D1 = ", model.D1.value)
print("D2 = ", model.D2.value)
print("A1 = ", model.A1.value)
print("A2 = ", model.A2.value)
print(model.obj())

Values of decision variables:
X1 =  0.0002926135688858585
X2 =  1.4603988954831293e-07
D1 =  142.85710070868643
D2 =  2.5909199324479745e-07
A1 =  258615.57088362065
A2 =  942.369949180316
785714.0261876862


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**